In [3]:
from google.colab import files
uploaded = files.upload()

Saving images_and_labels.csv to images_and_labels.csv


In [32]:
!pip install qiskit
!pip install qiskit_machine_learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 16.6 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512543 sha256=e4cd7eef308b8d38ef60d9ae67338444b66d547c623fd17fcd68994a6bdbb28a
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d51

In [4]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['images_and_labels.csv']))

In [1]:
import json
import os
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [9]:
X = df.drop(columns=['Label'])
y = df['Label']
y

,Label
0,0
1,0
2,0
3,1
4,1
...,...
4496,0
4497,0
4498,1
4499,0


In [209]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_reshaped = X_train.values.reshape(-1, 1, 28, 28)
X_test_reshaped = X_test.values.reshape(-1, 1, 28, 28)

In [210]:
import numpy as np
import torch
from torch.utils.data import Dataset

class NumpyDataset(Dataset):

    def __init__(self, x, y):
        if isinstance(x, np.ndarray):
            self.x = torch.from_numpy(x).to(torch.float)
        else:
            raise TypeError("x should be a numpy array")

        if isinstance(y, np.ndarray):
            if len(y.shape) == 1:
                y = y.reshape(-1, 1)
            self.y = torch.from_numpy(y).view(-1).long()
        else:
            raise TypeError("y should be a numpy array or pandas Series")

    def __getitem__(self, i):
        return self.x[i], self.y[i]

    def __len__(self):
        return len(self.y)


In [211]:
tr_data = NumpyDataset(X_train_reshaped, y_train.values)
test_data = NumpyDataset(X_test_reshaped, y_test.values)


In [221]:
from torch.utils.data import DataLoader
tr_loader = iter(DataLoader(tr_data, batch_size = 50, shuffle = True))
test_loader = iter(DataLoader(test_data, batch_size = 15, shuffle = True))

# print(next(tr_loader))

In [222]:
for batch_idx, (data, target) in enumerate(tr_loader):
    print(f"Batch {batch_idx + 1}:")
    # print("Data:", data)
    print("Data Shape:", data.shape)  # Print the shape of the data tensor
    # print("Target:", target)
    print("Target Shape:", target.shape)  # Print the shape of the target tensor
    # Optionally, break after a few batches to avoid printing too much
    if batch_idx == 4:
        break

Batch 1:
Data Shape: torch.Size([50, 1, 28, 28])
Target Shape: torch.Size([50])
Batch 2:
Data Shape: torch.Size([50, 1, 28, 28])
Target Shape: torch.Size([50])
Batch 3:
Data Shape: torch.Size([50, 1, 28, 28])
Target Shape: torch.Size([50])
Batch 4:
Data Shape: torch.Size([50, 1, 28, 28])
Target Shape: torch.Size([50])
Batch 5:
Data Shape: torch.Size([50, 1, 28, 28])
Target Shape: torch.Size([50])


In [223]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap,RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_algorithms.utils import algorithm_globals

In [224]:
def create_qnn():
    feature_map = ZZFeatureMap(2)
    ansatz = RealAmplitudes(2, reps=1)
    qc = QuantumCircuit(2)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn


qnn = create_qnn()

In [225]:
import torch
from torch import cat, no_grad, manual_seed
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    NLLLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
)
import torch.nn.functional as F

In [226]:
# initial_weights = 0.1 * (2 * algorithm_globals.random.random(qnn.num_weights) - 1)

In [227]:
import torch.nn as nn
# class Net(nn.Module):
#     def __init__(self, qnn):
#         super().__init__()
#         self.conv1 = Conv2d(1, 2, kernel_size=5)
#         self.conv2 = Conv2d(2, 16, kernel_size=5)
#         self.dropout = Dropout2d()
#         self.fc1 = Linear(256, 64)
#         self.fc2 = Linear(64, 2)  # 2-dimensional input to QNN
#         self.qnn = TorchConnector(qnn)  # Apply torch connector, weights chosen
#         # uniformly at random from interval [-1,1].
#         self.fc3 = Linear(1, 1)  # 1-dimensional output from QNN

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.max_pool2d(x, 2)
#         x = F.relu(self.conv2(x))
#         x = F.max_pool2d(x, 2)
#         x = self.dropout(x)
#         x = x.view(x.shape[0], -1)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         x = self.qnn(x)  # apply QNN
#         x = self.fc3(x)
#         x = torch.sigmoid(self.fc1(x))
#         x = torch.sigmoid(self.fc1(x))

#         x = self.qnn(x)
#         x = torch.linear(self.fc3(x))
#         return cat((x, 1 - x), -1)
class Net(nn.Module):
    def __init__(self, qnn):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, kernel_size=5)
        self.conv2 = nn.Conv2d(2, 16, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(16 * 4 * 4, 64)  # Corrected input size to match flattened conv output
        self.fc2 = nn.Linear(64, 2)  # 2-dimensional input to QNN
        self.qnn = TorchConnector(qnn)
        self.fc3 = nn.Linear(1, 1)  # Correct the input size here to match QNN output

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.qnn(x)  # Apply QNN
        x = self.fc3(x)  # Apply final linear layer and sigmoid
        return x
# class Net(nn.Module):
#     def __init__(self, qnn):
#         super().__init__()
#         self.fc1 = nn.Linear(784, 128)  # Input size matches the number of features in data
#         self.fc2 = nn.Linear(128, 2)    # Output size matches the number of classes
#         self.qnn = TorchConnector(qnn)
#         self.fc3 = nn.Linear(2, 1)      # Final layer to adjust the output size from QNN

#     def forward(self, x):
#         x = torch.sigmoid(self.fc1(x))
#         x = torch.sigmoid(self.fc2(x))
#         x = self.qnn(x)
#         x = torch.sigmoid(self.fc3(x))
#         return x

model_ = Net(qnn)

In [228]:
print(model)

Net(
  (conv1): Conv2d(1, 2, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(2, 16, kernel_size=(5, 5), stride=(1, 1))
  (dropout): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (qnn): TorchConnector()
  (fc3): Linear(in_features=2, out_features=1, bias=True)
)


In [229]:
optimizer = optim.Adam(model_.parameters(), lr=0.001)
loss_func = NLLLoss()

# Start training
epochs = 10
loss_list = []
model_.train()


for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (data, target) in enumerate(tr_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = model_(data)  # Forward pass
        print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(tr_loader)}], Model Output: {output}")
        loss = loss_func(output[:, 0], target)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss += loss.item()  # Accumulate loss

        print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(tr_loader)}], Loss: {loss.item():.4f}")

    # Calculate and store average loss for the epoch
    avg_loss = total_loss / len(tr_loader)
    loss_list.append(avg_loss)
    print("Training [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, avg_loss))


Epoch [1/10], Batch [1/72], Model Output: tensor([[-0.6714],
        [-0.6325],
        [-0.7030],
        [-0.6676],
        [-0.6430],
        [-0.6813],
        [-0.6425],
        [-0.6581],
        [-0.6847],
        [-0.6498],
        [-0.6421],
        [-0.6274],
        [-0.7287],
        [-0.7355],
        [-0.6884],
        [-0.6453],
        [-0.6642],
        [-0.7174],
        [-0.6772],
        [-0.6747],
        [-0.7218],
        [-0.7568],
        [-0.6789],
        [-0.6893],
        [-0.6741],
        [-0.6967],
        [-0.6677],
        [-0.7258],
        [-0.6473],
        [-0.7155],
        [-0.7045],
        [-0.6928],
        [-0.7080],
        [-0.7520],
        [-0.7187],
        [-0.7106],
        [-0.6785],
        [-0.7084],
        [-0.7425],
        [-0.6933],
        [-0.6740],
        [-0.6805],
        [-0.6868],
        [-0.7825],
        [-0.6830],
        [-0.6817],
        [-0.6634],
        [-0.6670],
        [-0.6468],
        [-0.7415]], grad_fn

In [206]:
torch.save(model.state_dict(), "model.pt")

In [64]:
qnn_eval= create_qnn()
model_eval = Net(qnn_eval)
model_eval.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [67]:
model_eval.eval()  # set model to evaluation mode
with no_grad():

    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model_eval(data)
        if len(output.shape) == 1:
            output = output.reshape(1, *output.shape)

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = loss_func(output, target)
        total_loss.append(loss.item())

    print(
        "Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%".format(
            sum(total_loss) / len(total_loss), correct / len(test_loader) / 15 * 100
        )
    )


Performance on test data:
	Loss: -0.5637
	Accuracy: 0.0%
